In [1]:
import Ipynb_importer
import numpy as np
import pandas as pd
from pandas import DataFrame
import utils,preprocess,merge
import time
import datetime
import os,gc
file_path='D:/360WiFi/'
raw_data_path='D:/360WiFi/FDDC_financial_data_20180613/'
split_file_path='D:/360WiFi/files/financial_data_new/'
input_path='D:/360WiFi/files/input/'
mid_path='D:/360WiFi/files/mid_file/'
other_path='D:/360WiFi/files/other_file/'


##------------------合表前，初筛列开关--------------------------------------------------------------
#资产负债表
del_col_Balance_Bank=['AE,AA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']
del_col_Balance_GB=['CAE,CAA,NCAE,NCAA,AE,AA,CLE,CLA,NCLE,NCLA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']
del_col_Balance_Insurance=['AE,AA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']
del_col_Balance_Securities=['AE,AA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']

#现金流量表
del_col_cashFlow_Bank=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']
del_col_cashFlow_GB=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']
del_col_cashFlow_Insurance=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']
del_col_cashFlow_Securities=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']


#利润表
del_col_Income_Bank=['SPEC_OR,AOR,SPEC_OC,AOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']
del_col_Income_GB=['SPEC_TOR,ATOR,SPEC_TOC,ATOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']
del_col_Income_Insurance=['SPEC_OR,AOR,SPEC_OC,AOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']
del_col_Income_Securities=['SPEC_OR,AOR,SPEC_OC,AOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']



del_list=[del_col_Balance_Bank,del_col_Balance_GB,del_col_Balance_Insurance,del_col_Balance_Securities,\
         del_col_cashFlow_Bank,del_col_cashFlow_GB,del_col_cashFlow_Insurance,del_col_cashFlow_Securities,\
        del_col_Income_Bank,del_col_Income_GB,del_col_Income_Insurance,del_col_Income_Securities]


##------------------合表后，删列/删行开关-------------------------------------

class_option='classifi_by_14L1'


##'END_DATE'会在函数中删除，'REVENUE_commom_income'作为特征,暂不删除

del_col=['PARTY_ID','EXCHANGE_CD', 'PUBLISH_DATE','END_DATE_REP',\
          'REPORT_TYPE','MERGED_FLAG', 'category',\
        'industryName_level_1','industryName_level_2','industryName_level_3','isNew','Mid_L2','Mid_L3',\
         'industryID_level_1','industryID_level_2','industryID_level_3',\
         'PUBLISH_DATE_y','PUBLISH_DATE_x','END_DATE_REP_x','END_DATE_REP_y'
        ]


'''
ticker_option的value中：第一项是起始时间，第二项是结束时间，第三项是要删的表，有四种选项['income','balance','cash','all']

'''
ticker_option={'601360':['2009-03-31','2018-03-31','all'],
               '600048':['2017-12-31','2018-03-31','all'],
               '600409':['2017-12-31','2018-03-31','all'],
               '600693':['2017-12-31','2018-03-31','all'],
               '600939':['2017-12-31','2018-03-31','all'],
               '600399':['2017-12-31','2018-03-31','all'],
               '600610':['2017-12-31','2018-03-31','all'],
               '600094':['2009-03-31','2009-09-30','all'],
              }

##--------------one_hot开关------------------------------------

oht_col=['TICKER_SYMBOL','FISCAL_PERIOD','industrySymbol_level_1','industrySymbol_level_2','industrySymbol_level_1',\
        'indexSymbol_level_1','indexSymbol_level_2','indexSymbol_level_3',\
        ]
# merge.preprocess(del_list)
# merge.generate_data_base()
# merge.merge_3file()
# preprocess.select_table(tabel,class_option,del_col,ticker_option)
# preprocess.make_label()

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from preprocess.ipynb
importing Jupyter notebook from merge.ipynb
----------------------------------------------------
Starting preprocessing....

loading file...
reading file:balance_sheet_Bank_new.csv
reading file:balance_sheet_GB_new.csv
reading file:balance_sheet_Insurance_new.csv
reading file:balance_sheet_Securities_new.csv
reading file:cashFlow_sheet_Bank_new.csv
reading file:cashFlow_sheet_GB_new.csv
reading file:cashFlow_sheet_Insurance_new.csv
reading file:cashFlow_sheet_Securities_new.csv
reading file:income_sheet_Bank_new.csv
reading file:income_sheet_GB_new.csv
reading file:income_sheet_Insurance_new.csv
reading file:income_sheet_Securities_new.csv
del some cols...
del col of df
del col of df
del col of df
del col of df
del col of df
del col of df
del col of df
del col of df
del col of df
del col of df
del col of df
del col of df
single sheet drop_duplicates...
preprosessing the normal ticker...
merging

c:\users\mr.handsome\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


merging CashFlow sheet.....

merging Income sheet.....

merging final sheet.....

preprosessing the normal ticker done!!!...

preprosessing the special ticker...
merging Balance sheet.....

merging CashFlow sheet.....

merging Income sheet.....

merging final sheet.....

preprosessing the special ticker done!!!...


KeyboardInterrupt: 

In [ ]:
'''
生成初步跑的大表（官方数据）

'''

# table=pd.read_csv(mid_path+'feature_importance_2.csv')
# table.get_dtype_counts()
# t=table.infer_objects().dtypes
# t[t=='int64']

# table['date']=table['year'].astype(str)+table['month'].apply(lambda x:"%02d"%x)
# table[['date','year','month']]

# T=pd.get_dummies(table,columns=['FISCAL_PERIOD'])
# T.get_dtype_counts()
# header=['date','FISCAL_PERIOD_3', 'FISCAL_PERIOD_6', 'FISCAL_PERIOD_9', 'FISCAL_PERIOD_12']
# T=T[header+list((set(T.columns)-set(header)))]
# T
# T.get_dtype_counts()

# dangji=pd.read_csv(mid_path+'table_6_file_label.csv')
# dangji.rename(columns={'label':'Revenue_now'},inplace=True)

# merge=pd.merge(T,dangji[['TICKER_SYMBOL','year','month','Revenue_now']],how='left',on=['TICKER_SYMBOL','year','month'])
# rr=merge[['TICKER_SYMBOL','year','month','Revenue_now','label']]
# rr[rr.year!=2018].to_csv(other_path+'check_revenue_prerevenue.csv',index=False)

#merge.drop(['year','month','TICKER_SYMBOL','REVENUE_commom_income'],axis=1,inplace=True)
# table
#merge.get_dtype_counts()
#merge[merge.date!='201803'].to_csv(other_path+'feature_importance_2_update.csv',index=False)
preprocess.make_label()

In [23]:
# table=pd.read_csv(mid_path+'table_6_file_label_new.csv')
columns=table.columns
columns[columns.str.contains('PARTY_ID')][0]

'PARTY_ID'

In [47]:
def C_N_dic():


    sheetname=['bs_indu','bs_bank','bs_secu','bs_insu','is_indu','is_bank','is_secu','is_insu','cf_indu','cf_bank','cf_secu','cf_insu']

    E_col=[]
    C_col=[]
    source=[]
    for name in sheetname:
        C_N=pd.read_excel(other_path+'Data Dictionary_5.29.xlsx',sheetname=name)
        t=C_N.drop([0,1,2,3])
        E_col.extend(t.iloc[:,0].values)
        C_col.extend(t.iloc[:,2].values)
        source.extend([name for i in range(len(t.iloc[:,2].values))])

    C_N=DataFrame({'E_col':E_col,'C_col':C_col,'source':source})
    C_N.drop_duplicates(subset=['E_col'],inplace=True)

    ##注意一下COGS

    #选取第几列
    #t.iloc[:,[0,2]]

    C_N.to_csv(file_path+'C_N_dic.csv',encoding='gbk',index=False)
    
    return 

c:\users\mr.handsome\appdata\local\programs\python\python36\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [24]:
pp=pd.read_excel(other_path+'[New] Macro&Industry_20180613.xlsx',sheetname='指标信息')
pp.INDIC_ID=pp.INDIC_ID.astype(str)
pp=pp[['INDIC_ID','NAME_CN']]
pp.rename(columns={'INDIC_ID':'trains_col'},inplace=True)

# dic1=pd.read_csv(other_path+'C_N_dic.csv',encoding='gbk')
# dic1

# dic2=pd.read_csv(other_path+'C_N_Trans.csv',encoding='gbk')
# dic2


# pd.merge(dic1,dic2,how='left',on=['E_col']).to_csv(other_path+'CN_dic_3_sheet.csv',index=False,encoding='gbk')


# table=pd.read_csv(mid_path+'table_6_file_label_new.csv')
# table2=pd.read_csv(other_path+'CN_dic_3_sheet.csv',encoding='gbk')
# ttt=DataFrame({'trains_col':table.columns})

ww=pd.merge(ttt,table2[['C_col','trains_col']],how='left',on=['trains_col'])

pd.merge(ww,pp,how='left',on=['trains_col']).to_csv(file_path+'C_N_6_sheet.csv',index=False,encoding='gbk')

c:\users\mr.handsome\appdata\local\programs\python\python36\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [ ]:

def get_feature_data():
    '''
    生成初步跑的大表（按要求筛掉部分宏观信息和company_operation 信息）,feature_importance.csv
    
    '''
    table=pd.read_csv(mid_path+'table_6_file_label_new.csv')
    table=preprocess.select_table(table,class_option,del_col,ticker_option)

    table.drop(table.columns[355:427],axis=1,inplace=True)
    table.drop(['industrySymbol_level_1','industrySymbol_level_2','industrySymbol_level_3',\
                'indexSymbol_level_1','indexSymbol_level_2','indexSymbol_level_3','classifi_by_14L1',],axis=1,inplace=True)
    table.to_csv(other_path+'feature_importance.csv',index=False)
    print('the feature_importance.csv  shape:',table.shape)
    return 

check_revenue()

In [ ]:
##探究主表的列类型


#A.columns[A.columns.str.contains('REVE')]#列中包含某字符的列名
#A.get_dtype_counts() 列的类型计数

#A.infer_objects().dtypes#具体到每列的类型
#t#[t=='object']


In [ ]:
# def compare(df1,df2):
#     '''
#     比较表间ticker异同
    
#     '''
#     print(len(df1.TICKER_SYMBOL.unique()))
#     print(len(df2.TICKER_SYMBOL.unique()))
#     print(set(df1.TICKER_SYMBOL.unique())-set(df2.TICKER_SYMBOL.unique()))
#     print(set(df2.TICKER_SYMBOL.unique())-set(df1.TICKER_SYMBOL.unique()))
    
    
# compare(balance_sheet_Securities_old,income_sheet_Securities_old)

In [ ]:

    ##1.market data 检查value 和申万知否一致
    
#market_data=pd.read_excel(raw_data_path+'[New] Market Data_20180613.xlsx')
#market_data.TICKER_SYMBOL.unique().shape#(3675,)
# test_180330=market_data[market_data['END_DATE_']=='2018-03-30'][['TICKER_SYMBOL','MARKET_VALUE']]
# check_180830=pd.read_csv(file_path+'Uer_market_2018330.csv')
# check_180830=check_180830[['ticker','negMarketValue','marketValue']].rename(columns={'ticker':'TICKER_SYMBOL'})
# pd.merge(test_180330,check_180830,how='left',on=['TICKER_SYMBOL']).to_csv(file_path+'check_market_values.csv',index=False)
#================================================================
    ##2.从三表中读取12表

# balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
# cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
# income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB=utils.loadingFile(0)

#================================================================
    ##3.将12个表写到csv中，方便今后读取

# file_name=['balance_sheet_Bank','balance_sheet_Securities', 'balance_sheet_Insurance','balance_sheet_GB',\
# 'cashFlow_sheet_Bank','cashFlow_sheet_Securities','cashFlow_sheet_Insurance','cashFlow_sheet_GB',\
# 'income_sheet_Bank','income_sheet_Securities','income_sheet_Insurance','income_sheet_GB']

# file_df=[balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
# cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
# income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB]

# for df,name in zip(file_df,file_name):
#     df.to_csv(file_path+'files/financial_data_new/%s.csv'%name)
#============================================================================
    ## 4.在csv文件中补全00000

# A=pd.read_csv(file_path+'sumbit_5_31.csv',encoding='gbk')
# A.TICKER_SYMBOL=A.TICKER_SYMBOL.apply(lambda x:'\t%06d'%x)
# A.to_csv(file_path+'523.csv',index=False)

# A=pd.read_csv(file_path+'sumbit_5_31.csv',encoding='gbk')
# A.TICKER_SYMBOL=A.TICKER_SYMBOL.apply(lambda x:'%06d'%x)
# #A.to_csv(file_path+'523.csv',index=False)
# for a in A.TICKER_SYMBOL:
#     print(a)

#==============================================================
    ## 5.补全下载数据的ticker
# file_df=pd.read_csv(file_path+'cashflow_daily_all.csv')
# #file_df=pd.read_hdf(file_path+'CAI.h5',key='data')
# ticker_df=pd.read_csv(file_path+'ticker_list_new.csv')
# file_tickers=file_df.ticker.unique()
# ticker_tickers=ticker_df['0'].values
# dfii_list=[x for x in ticker_tickers if x not in file_tickers]
# DataFrame(dfii_list).to_csv(file_path+'DIFF-个股资金日流向.csv',index=False)


In [ ]:

#===================================================================================    
    ##三表ticker检查
'''
compared with 529data:add 5 more tickers
 array([300747], dtype=int64),
 array([601066], dtype=int64),
 array([603650], dtype=int64),
 array([603666], dtype=int64),
 array([603693], dtype=int64)]

'''


# Balance=[balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old]
# del balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old
# Balance

# Balance_df=pd.concat(Balance)
# Balance_df.TICKER_SYMBOL.unique()#3551

# Cash=[cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old]
# del cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old
# Cash

# Cash_df=pd.concat(Cash)
# Cash_df.TICKER_SYMBOL.unique()#3551

# Income=[income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old]
# del income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old
# Income

# Income_df=pd.concat(Income)
# Income_df.TICKER_SYMBOL.unique()#3551



# print('the shape of Balance_df is',Balance_df.TICKER_SYMBOL.unique().shape)#(3556,)
# print('the shape of Cash_df is',Cash_df.TICKER_SYMBOL.unique().shape)#(3556,)
# print('the shape of Income_df is',Income_df.TICKER_SYMBOL.unique().shape)#()(3556,)


# tiker_list=pd.read_csv(file_path+'ticker_list.csv')
# tiker_list['0'].unique()


# #比较三大表中ticker是否相同
# A1=set(Balance_df.TICKER_SYMBOL.unique())
# A2=set(Cash_df.TICKER_SYMBOL.unique())
# A3=set(Income_df.TICKER_SYMBOL.unique())

# print('A1==A2',A1==A2)
# print('A1==A3',A1==A3)
# print('A2==A3',A2==A3)

#DataFrame(sorted(Balance_df.TICKER_SYMBOL.unique())).to_csv(file_path+'ticker_list_new_613.csv',index=False)

#===============================================================   
    ##检查529和613文件中ticker异同
# ticker_529=pd.read_csv(file_path+'ticker_list_new.csv')
# ticker_613=pd.read_csv(file_path+'ticker_list_new_613.csv')
# [x for x in ticker_613.values if x not in ticker_529.values]